The Lagrangian Averaged Vorticity Deviation (LAVD) is computed from the vorticity $ \omega $ as:

\begin{equation}
\mathrm{LAVD}_{t_0}^{t_N}(\mathbf{x}_0) = \dfrac{1}{t_N-t_0} \sum_{i = 0}^{N}| \omega(\mathbf{x}(t_i)) - \overline{\omega}(t_i) |
\end{equation}

where $ \omega(\mathbf{x}(t_i)) $ indicates the vorticity at $ \mathbf{x}(t_i) $. $ \overline{\omega}(t_i) $ is the spatial average of the vorticity at time $ t_i $.

| Name | Type (Shape) | Description |
| --- | --- | --- |
| omega | array (Nt, Ny, Nx, Nz) | vorticity $ \omega $ along trajectories|
| lenT | float | $ t_N-t_0 $|
| LAVD | array (Ny, Nx, Nz) | $ \mathrm{LAVD}_{t_0}^{t_N}(\mathbf{x}_0) $|

In [1]:
import sys, os

# get current directory
path = os.getcwd()

# get parent directory
parent_directory = os.path.sep.join(path.split(os.path.sep)[:-3])

# add utils folder to current working path
sys.path.append(parent_directory+"/subfunctions/utils")

In [ ]:
# Import numpy
import numpy as np

# Import function which calculates gradient of velocity
from ipynb.fs.defs.gradient_velocity import gradient_velocity

# import math tools
from math import sqrt

# import progress bar
from tqdm.notebook import tqdm

In [2]:
def _LAVD(omega, lenT, omega_avg = None):
    
    omega = omega.reshape((omega.shape[0], 3, -1))
    
    if omega_avg is None:
    
        omega_avg = [] # list (Nt,)
        for t in range(omega.shape[0]):
            omega_avg.append(np.mean(omega[t,:,:], axis = -1))

    omega_avg = np.array(omega_avg)
    LAVD = np.zeros((omega.shape[2]))

    for t in tqdm(range(omega.shape[0])):
        for i in range(omega.shape[2]):
            omega_dif = omega[t,:,i]-omega_avg[t,:]
            LAVD[i] += sqrt(omega_dif[0]**2+omega_dif[1]**2+omega_dif[2]**2)
    
    return LAVD/lenT